In [1]:
from __future__ import print_function
import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
text=(open("/Users/maria/github/Dataset.txt", encoding = "utf-8-sig").read().lower())
print("____________________________________________________________")
words = keras.preprocessing.text.text_to_word_sequence(text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
print('total words:', len(words))
print(words)
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

____________________________________________________________
total words: 1741
['this', 'past', 'week', 'was', 'the', 'conclusion', 'of', 'our', 'year', 'long', 'event', 'iap', 'we', 'want', 'to', 'thank', 'all', 'participants', 'and', 'company', 'representatives', 'for', 'showing', 'interest', 'in', 'helping', 'our', 'students', 'further', 'develop', 'their', 'professional', 'careers', 'through', 'undergraduate', 'research', 'ece', 'uprm', 'iap', 'research', 'today', 'some', 'of', 'our', 'members', 'volunteered', 'to', 'go', 'and', 'teach', 'computation', 'concepts', 'to', '7th', 'grade', 'students', 'from', 's', 'u', 'aquilino', 'caban', 'school', 'in', 'aguada', 'pr', 'cahsi', 'uprm', 'wants', 'to', 'thank', 'all', 'of', 'those', 'involved', 'in', 'this', 'outreach', 'effort', 'cahsi', 'uprm', 'ece', 'outreach', 'last', 'week', 'our', 'department', 'offered', 'the', 'focus', 'areas', 'orientation', 'for', 'electric', 'and', 'computer', 'engineering', 'students', 'and', 'professors',

In [3]:
# cut the text in semi-redundant sequences of maxlen words
maxlen =40
step = 3
sentences = []
next_words = []
for i in range(0, len(words) - maxlen, step):
    
    sentences.append(words[i])
    next_words.append(words[i])
print('nb sequences:', len(sentences))


nb sequences: 567


In [4]:

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence.split()):
        x[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1


Vectorization...


In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(words))))
model.add(Dense(len(words), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = ''
        for i in range(7):
            random_index = random.randint(0, len(words) - 1)
            sentence += " " + words[random_index]
        #sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(30):
            x_pred = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence.split()):
                
                x_pred[0, t, word_indices[word]] = 1.
                

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            generated += " "+ next_word
            sentence += " " + next_word

            sys.stdout.write(" " + next_word)
            sys.stdout.flush()
        print()
        
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
567/567 [==============================] - 4s 7ms/step - loss: 5.0362

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " a provide icom drone the school students"
 a provide icom drone the school students to to to to to to to to to to to to to to to to to to to to to to to to to to a to to to
----- diversity: 0.5
----- Generating with seed: " age time their google at cahsi in"
 age time their google at cahsi in department to ece a to from to visited if activity to event a celebrate to to a about microgrid to to to a to high and to place it research
----- diversity: 1.0
----- Generating with seed: " conference girl de students had recent conference"
 conference girl de students had recent conference talent directors careers meeting which very with future interested conference engineering and uprm october performed to graduation to after her dr and place uprm casa november thomas it undergraduate communication
----- diversity: 1.2
----- Ge

 the funded relating the also professors thank the to to to to to to to to to to to the to to to to to to to to to to to to to to the to to
----- diversity: 0.5
----- Generating with seed: " high de conference henaac we members most"
 high de conference henaac we members most institution friends winning our a student electrical and introduced very his department recent on led orientation and georgia will lianne puerto a rivas and 21 203 aguada research they came
----- diversity: 1.0
----- Generating with seed: " arg sh the orientation them sponsored team"
 arg sh the orientation them sponsored team maria by most helps are nayda came help colegiales 2017 students our nayda through staff nayda to award velez regulation of spanish 2018 observatory his department solar up nsrec event
----- diversity: 1.2
----- Generating with seed: " de bears for he your electrical auditorium"
 de bears for he your electrical auditorium short radiation conference be department s science observatory concept

 or to in prestigious work the teach who eventful course computer magnetism to colegio gonzalez they computer award november very hardware verde national future communication aguada experience high october rico” led technical up with computer advisory biology
----- diversity: 0.5
----- Generating with seed: " have want careers programs our student spring"
 have want careers programs our student spring to nayda to his a to orientation microgrid rico orientation event and research such department concepts to recognized his today as have have to to will and to 2018 such
----- diversity: 1.0
----- Generating with seed: " miguel google they in colegio this electronics"
 miguel google they in colegio this electronics to school meeting to about to it came on iap to her department recognized 00 research rico project a meeting or more to after to to observatory to to came
----- diversity: 1.2
----- Generating with seed: " department you good also which microgrid uprm"
 department you good also 

 an workshops in and kristalys and students have an semester after undergraduate reching for school october cahsi electrical interested orientation today laboratory and thank more had microgrid first gave couvertier he nayda dr an this at semester
Epoch 19/60
567/567 [==============================] - 4s 7ms/step - loss: 5.3545

----- Generating text after Epoch: 18
----- diversity: 0.2
----- Generating with seed: " of concepts time 17th or also if"
 of concepts time 17th or also if and and and and and and and and and and and and and and and and and and and of of and and and and and of our and and
----- diversity: 0.5
----- Generating with seed: " working people of their will doing the"
 working people of their will doing the semester undergraduate to research student this be to thank such to to a to a her to to an to to a to be have to to to from uprm
----- diversity: 1.0
----- Generating with seed: " and offers info university unique paper on"
 and offers info university unique paper

 interested masters were boy faculty radiation with from observatory as congratulations as engineering 13th iap who the had at to rico it as an high at an student microgrid high interested introduced student academic this verde november
Epoch 25/60
567/567 [==============================] - 4s 7ms/step - loss: 5.4574

----- Generating text after Epoch: 24
----- diversity: 0.2
----- Generating with seed: " in our teach last soon now and"
 in our teach last soon now and 2018 2018 meeting 2018 2018 2018 2018 2018 2018 2018 project 2018 puerto puerto interested 2018 2018 undergraduate 2018 2018 2018 interested 2018 puerto puerto by 2018 2018 2018 2018
----- diversity: 0.5
----- Generating with seed: " is department their soon in our electrical"
 is department their soon in our electrical their and of their their and of and of and for of and of their and their and their and their their and for their and their and their students
----- diversity: 1.0
----- Generating with seed: " students off

 received 14 work able to service for november have project an tech uprm on at such interested concepts a via on an to dr had careers at on have his undergraduate today student as y on more
Epoch 31/60
567/567 [==============================] - 4s 7ms/step - loss: 4.3631

----- Generating text after Epoch: 30
----- diversity: 0.2
----- Generating with seed: " your biology have at is ex island"
 your biology have at is ex island communication executive executive executive executive executive we executive executive communication executive we we we executive executive we executive we we we we executive this executive we we we we we
----- diversity: 0.5
----- Generating with seed: " for in our second is category for"
 for in our second is category for our our of of of students and and our of of of of their and and and of of and students and of of and their and and and of
----- diversity: 1.0
----- Generating with seed: " orientation play 3pm therefore with event candidates"
 orientation pl

----- Generating with seed: " first over nayda’s would mayaguez from drone"
 first over nayda’s would mayaguez from drone in sebastian we prepares in uprm the this an uprm organization this in economic the award next the future in communication led science an we award the projects the in
----- diversity: 1.0
----- Generating with seed: " harris andrade gather on him with the"
 harris andrade gather on him with the electrical it visit this we so support receiving computer speak georgia professors campus star ao award october corporation sunday november activity have pr an before hourofcode several miguel studying uprm
----- diversity: 1.2
----- Generating with seed: " engineering observatory six this year residence the"
 engineering observatory six this year residence the a best students will thank and to is their microgrid students of their to of as and students student students orientation future y of orientation be congratulations an puerto their
Epoch 38/60
567/567 [================

 professors and the orienting programs so careers spanish award held professors have boston member the arg outreach an having we marrero the marrero appointed six biology learning documentary place long outstanding in to have organization july icom
----- diversity: 1.0
----- Generating with seed: " convene ildefonso colaborating grade students scientific students"
 convene ildefonso colaborating grade students scientific students in exposed house lives adrian have deadline coop learning agustin helps next short have presenting appointed relating national miguel two institution volunteered was through some several inspire three staff agustin
----- diversity: 1.2
----- Generating with seed: " colegio electrical share 26th a from closed"
 colegio electrical share 26th a from closed science biology interested verde aguada an exposed most relating performed nuclear secure see held aguada an category school we computer abilities all executive short e learning helps solar economic july
Epoch 

 annual communication on electrical 15 outreach full a a a a a their a november to a a have to a have have event a school a to to a to school november a a a a
----- diversity: 0.5
----- Generating with seed: " them school concepts of cahsi of drone"
 them school concepts of cahsi of drone department an event uprm to to celebrate their november as eventful gmis research january verde week agustin six of reus have to their y professors uprm prototyping a to puerto
----- diversity: 1.0
----- Generating with seed: " proud students of give we 2018 achievement"
 proud students of give we 2018 achievement laboratories they google had work how event the do to an thomas icom colegio nuclear department advancement experience 21 be internships member radiation such rico” google to available talked spanish
----- diversity: 1.2
----- Generating with seed: " research hardware student best resilience recent stood"
 research hardware student best resilience recent stood next interested all scouts febr

 concepts thank scouts be director research that a a a a department by it microgrid microgrid a celebrate a a a iap semester semester microgrid their a a celebrate january to by a by a by a
----- diversity: 1.0
----- Generating with seed: " they the of mixed the swe this"
 they the of mixed the swe this excellence development helps fabio atmospheric studying to current andrade through they before some google hardware activity achievement secure to damaged training speak scouts perspective hourofcode damaged technical ieee month rivas
----- diversity: 1.2
----- Generating with seed: " had vacation celebrate ece and proud swe"
 had vacation celebrate ece and proud swe peers engineering velez will month ece rico after will magnetism rico had is rico meeting undergraduate thank thank microgrid project our his observatory had meeting dr undergraduate nayda by they
Epoch 56/60
567/567 [==============================] - 4s 7ms/step - loss: 4.2589

----- Generating text after Epoch: 55
----- d